In [ ]:
%pip install numpy opencv-python matplotlib


In [ ]:
# 🔧 Importación de librerías necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt


# 📷 Cargar una imagen en escala de grises
img = cv2.imread('../datos/images.jpg', cv2.IMREAD_GRAYSCALE)

# 🖼️ Visualizar la imagen original
plt.imshow(img, cmap='gray')
plt.title('Imagen Original')
plt.axis('off')
plt.show()



In [ ]:
# 🧠 Implementación manual de la convolución 2D
def aplicar_convolucion(imagen, kernel):
    h, w = imagen.shape             # Dimensiones de la imagen
    kh, kw = kernel.shape           # Dimensiones del kernel
    pad_h, pad_w = kh // 2, kw // 2 # Padding necesario

    # 📦 Padding con ceros alrededor de la imagen
    imagen_padded = np.pad(imagen, ((pad_h, pad_h), (pad_w, pad_w)), mode='constant', constant_values=0)
    salida = np.zeros_like(imagen)

    # 🔁 Aplicar el kernel sobre cada píxel
    for i in range(h):
        for j in range(w):
            region = imagen_padded[i:i+kh, j:j+kw]
            salida[i, j] = np.clip(np.sum(region * kernel), 0, 255)
    
    return salida.astype(np.uint8)


In [ ]:
# ✴️ Filtro de enfoque (sharpening)
kernel_sharpen = np.array([[0, -1, 0],
                           [-1, 5,-1],
                           [0, -1, 0]])
img_sharp = aplicar_convolucion(img, kernel_sharpen)

# 🌫️ Filtro de suavizado (blur promedio)
kernel_blur = np.ones((3,3), np.float32) / 9
img_blur = aplicar_convolucion(img, kernel_blur)

# 🧭 Filtro de detección de bordes con Sobel
kernel_sobel_x = np.array([[-1, 0, 1],
                           [-2, 0, 2],
                           [-1, 0, 1]])
kernel_sobel_y = np.array([[-1, -2, -1],
                           [0,  0,  0],
                           [1,  2,  1]])

# Aplicación de ambos kernels en X e Y
img_sobel_x = aplicar_convolucion(img, kernel_sobel_x)
img_sobel_y = aplicar_convolucion(img, kernel_sobel_y)

# 🔍 Magnitud del gradiente para combinar direcciones
img_edges = np.sqrt(img_sobel_x**2 + img_sobel_y**2)
img_edges = np.clip(img_edges, 0, 255).astype(np.uint8)


In [ ]:
# 🛠️ Aplicar los mismos kernels con OpenCV
img_cv_sharp = cv2.filter2D(img, -1, kernel_sharpen)
img_cv_blur = cv2.filter2D(img, -1, kernel_blur)
img_cv_edges = cv2.filter2D(img, -1, kernel_sobel_x) + cv2.filter2D(img, -1, kernel_sobel_y)


In [ ]:
# 📑 Función para mostrar imágenes en paralelo
def mostrar_comparacion(titulo, img1, img2):
    fig, axs = plt.subplots(1, 2, figsize=(10, 4))
    axs[0].imshow(img1, cmap='gray')
    axs[0].set_title(f'{titulo} - Manual')
    axs[1].imshow(img2, cmap='gray')
    axs[1].set_title(f'{titulo} - OpenCV')
    for ax in axs: ax.axis('off')
    plt.tight_layout()
    plt.show()

# 🔍 Mostrar comparaciones
mostrar_comparacion('Sharpening', img_sharp, img_cv_sharp)
mostrar_comparacion('Blur', img_blur, img_cv_blur)
mostrar_comparacion('Bordes', img_edges, img_cv_edges)


In [ ]:
# 🎚️ Función dummy para el slider
def nothing(x):
    pass

# 🪟 Crear ventana redimensionable y sliders
cv2.namedWindow("Filtro Interactivo", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Filtro Interactivo", 700, 400)
cv2.createTrackbar("K1 (Centro)", "Filtro Interactivo", 1, 10, nothing)

while True:
    k1 = cv2.getTrackbarPos("K1 (Centro)", "Filtro Interactivo")

    # 🧩 Kernel ajustable de realce
    kernel = np.array([[0, -1, 0],
                       [-1, 4 + k1, -1],
                       [0, -1, 0]])

    result = cv2.filter2D(img, -1, kernel)

    # 🎨 Convertir a BGR para dibujar texto en negro
    img_bgr = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    result_bgr = cv2.cvtColor(result, cv2.COLOR_GRAY2BGR)

    # 🖍️ Texto suave, pequeño y de color negro
    font_scale = 0.6
    color = (0, 0, 0)  # Negro
    thickness = 1
    position = (10, 20)

    cv2.putText(img_bgr, 'Original', position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA)
    cv2.putText(result_bgr, 'Editada', position, cv2.FONT_HERSHEY_SIMPLEX, font_scale, color, thickness, cv2.LINE_AA)

    # ➕ Combinar ambas imágenes horizontalmente
    combinada = np.hstack((img_bgr, result_bgr))

    # 🖼️ Mostrar la ventana
    cv2.imshow("Filtro Interactivo", combinada)

    # ⏹️ Salir con ESC
    if cv2.waitKey(1) & 0xFF == 27:
        break

cv2.destroyAllWindows()